<a href="https://colab.research.google.com/github/dharshriya15/ML-Projects/blob/main/Exercise_Grouping_and_Sorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T082146Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D07cd2a7ae85aae6fe5e6a084f4327f2841c183db91e03f63da9e1191e4c2ac84d756802ba5bbb649877016f3b01e730e2ee34e9b0cab9f7ab624b08d2a28d625aa054db71aeccc0c27e1c2d9e18231b26c7bd6ba9e0fe05d3c6393233b1fa02f51616072bb50bf2184114304cd7a918e7ffc68abe2d29836e4c6d2e58461bb50437f4b8c5cb8d73cb58fd415bcf6d656f6f342308500bff61db21fdd72185b4587ff2bbd54667ebd9b32799b4093bd132c48ec15d03f02ea7f45faba8e94c7332b15d9bd64162a0205a75d4b98cfe9bbe409704266db15c72ced4bb1d9cbbe5176edd5e0929156fbae0e53a47e2c919aff20f0170c11435ebba0abef55fc5c7f,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T082146Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D809d53efe9333f77f29c82c2d33e0fee3e9c1919d8320eb5a815c29bc336ac91cedc8ab20cd0488482ce29c29060891dd27db4a22b400bc1b08c99c7310aaab1ca134847c07d0e235a28130fd1bf3b6e0fbecb00d9d481eb9eea10ea0ae692f34928d3e7e1b4f5f6f8aa180627823a69eb9f446c51a3eb4b144f93df4bff5e3c0325b95b46d4e5896f71fa908481d58303c5ddff9dd6a4157af92480c37c155d4e0c0909f0ad0e046bfb1eafd363eb98978cef0f6d443b8a4c6ad8937a687f1c56cec6c12cb1be81e13d22d7bb4ce1855e6347798038d8d57642d7b406a5f41b24458f7d3de27c2a5e07620cf7a71f78d7b49cb2ab3391b081463532d0b1e98c,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T082146Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8cfc264eba510472678351c4f4cfbb4f177b5107d6e2a28a5f1351f6ab17f71ceceaee5f64480b33c9de371cc1a2629c2b4cbe0b7b2193af2068eb3c03036a7cb79fd4f649ff2d650e71734a7e5303f144c3dfee37ebbf68e0cc63a1734fa00ec0d83a0ec25d393bdeedab509bb4d1abdbd6a93021bbd6e1bb85b1cf59dc11174255c300e397217623f72d5670ffb28141a2586d36c35dd8699d217624c54ee6a0685febcb0604773e4781deff3baf1d9f3da1b1e9dd5e2e01471765ac450c9304d787f30d1d2b920b266b4ab8ab98a241f8dd216e43984cfa4582f632de7c8c4a3c18976c7eb6d53fd35a484988a170650ad3bb28817bd340234c9a7ec11fa0,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T082147Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D800f0ee03d5e28ca49b4761e949f8cfcd4242aff64004501931ce2e24ffbad149d848d06c4669521bc641a804dc20d4609739a8c760dbbd9e1e933e8efb2430520631f1be32448d569e75c0f090af38c6ee029d29e2fa604b81309ce39a09713edd64f2ace1eaec948df1e062084f18d001f49e930ae317c62d2f645ed6c1ddb81b13ab7de44768459f88483c0750cd86363f08bfff61282427fe9d62d2a12c8d44c5446a576e140c23f80a18a2af75ae0dab052a22a943b1c75e4b417407ab066286382b314ed44be47e6387373fcdfe3db90eb48f5eafd94bb9396f067d7ca7be2392069014e365e7a66cd2f286d5a566473718d9bb3ea81d451d28ec7ad70,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T082147Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9e28ebcdb239d36156982d03283eaff39f48c4de8c0d6f817aca78b7c95a68d54fa3e05f68df06258eae60bedaabe1c892d59012b56cbcdbc43a6b4a79429c20a5a33504ae95ed84cb5deb8f3c8371d996b9f28a2a64fea67c5cfd315dabce22c4548a562af9bd9f482a613d6fbbed2219fea4421401e286280a1929db0dd9f3758f040c88364c61b358eb5d1530dd97460702258e719cc80d6e762d706af65815db6cc9b00d5c44d3a0b277c37629f353d4e3b4d045aad4ea0e47fc36814f236e0e4ef094277c6c7f3f4ec9d957385abcceab86b8121d69c171fdf0c18aefdb222f6ac70f047747a07e90164f196767e0cc63cdf7a0daf9c90333d5dbaebeec,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T082147Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D98fac1e4a03b91bbeebbe4bd15c6677fc3b6fd693ebad3127c50c950ec6b0d344a21edf84895017b5feea88cb6763b41daf478f784fea7a27e4314ba7fa287a9679578b90f01c32a5e82c61a076d6e2077398a021c65348eb8848b72d0353078b326553aac9ef365b39fc2f70fe283dbd1600f58224ad551e7e8086a9a01e2c0c11cc3cbb4f62b4c2d00e0dbc6474bb92a4850d70de226930f6d043ec67841710e3f65f89d9e606ebcf8896c9ec076fafed92f2829675b03111ca723755a2488b691e0c676dac23b78d723a997af5be9c95429e2f42eb80f49852476aeb88b8994e0b0b357654771bfdab0bbde340e7cf2bd01e565007c62ba0fa38a4e380921,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T082147Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db38406f3bcfd1e36cec9be450d55e8f668acfc2c54734c3779488f764764686a3bcdf1bf00fb51ff55f8b5d1301e7e0c8cf36383a722719e3c75d134622d383e5d501da9b5a3f28d35a06dd3561435fce2745a35d1f58421785b68965d3e71e7bf57b278f9a37e81d22ba00fdf9518084c3dde1198e1ab47dd482ad4e84f221a204b05628f1804d65124f227ca3da3979f8a272187a9eac6afd1cbcc14ff1dc16568f8ff7783f8b57dccd057511850a8f946f0d4e140e2cf1bf96ab5ce74b91ab57481edef05e813f08ed979b05ecf3309f826099bd651d57ce59cf4f4f6b29ec2cb8251ee7e0361c0695bf5579d8b7ea4d6f9f99b86ee8cf815ec65fa048541,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T082147Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3d71426571f70661a742f731375bb1bd48873f56558e0b55ad82cdcec90ad292a1cfe4986b8270cabb46050ddea2d3a1bbd103111cca2ca9aa478dad59838bfd4a97aadb774b0eeb3ca168b4b9384d5cf3bf237d347659d1ced186d5b87ab6721bd0fe1d8cd2f0018ef345ad53003ec67c1c42d0722992cc16b1f2271093eca7214193feb1021ccaa49d94fffb32f19b1210dc4da1817193455f07fc4f9ce6c70927cd14843eefd004895def9a32b5a316f0d1957853cc1f7ae0c8ba005006f3ae7951d4c8366e00e0029b13e682992403c3199e03286e1dfe1fb8eb52450102fdd8ddbf79a559661352f1632e0772d21193154996fd26e8ccac31d298fe5b14,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T082147Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2adca9d5aad01e8c3ce3beda5c6156dc559a1fd69abe4456ed59250619716239f6e5ff19e1f50db0e8602bfb3ee1d0897b1b44a8cfb1b82dc02d9844ab3df0f3cb040d67df358c067fadb50f31821212fc8518ee2ba407caf0a1ba948fdd378333aaccd375e00216e0ba1e6658d133196cc40c2801b5a363ac1c288f5244076379da0a46d6a6f150f83cbd364c22c6615af615d20254b99a2cac662ad7a887fe753ae4225f07ed79bad32309378e4cc6e9c045d4a0944a77a76beadeaf13f08e85781cd77aa0190ba403fa2c30fa1d43856cada2b7a5af7f47c27fc81c472970171ab9d1332ab932f04fd7448e62d2e331a99f4ed305fe0a40d0846e2cf3ba3c,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T082147Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D716b272c567331d662bb99eedb17377130fe6eb8f1aebfd1fdf1e50b3a7d565cfdc4abfc86c4dc900508bb7a29229bf4ad90fc01921250de620ac9517baea74f6876cf97c237d45870d813726bc3ff5954a677dd57597efe784c3e006418ca7f9d07c5b1d651c56a466a222296be8e3717abe22d4b79a26ad5ab4c0841a8ffa3d88f5627b0169c420a8c921976e97e72910aeb77eef1e6baf6644c7a8fbd54881f742b43e528f78d70cb55df9ccc450615044ad714e48e9f7cba392c22890989f82c425d39f0bca2c84a3b56b7f0a738f4504c60a55a13c2a9f892ced7b05c4259c6ccc33c256f1c62dea25d57e5b804c6b89a46374ff9dbe5a2fc57c3a0a61a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/grouping-and-sorting).**

---


# Introduction

In these exercises we'll apply groupwise analysis to our dataset.

Run the code cell below to load the data before running the exercises.

In [ ]:
import pandas as pd

reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
#pd.set_option("display.max_rows", 5)

from learntools.core import binder; binder.bind(globals())
from learntools.pandas.grouping_and_sorting import *
print("Setup complete.")

Setup complete.


# Exercises

## 1.
Who are the most common wine reviewers in the dataset? Create a `Series` whose index is the `taster_twitter_handle` category from the dataset, and whose values count how many reviews each person wrote.

In [ ]:
# Your code here
reviews_written = reviews.groupby('taster_twitter_handle').taster_twitter_handle.count()

# Check your answer
q1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 


```python
reviews_written = reviews.groupby('taster_twitter_handle').size()
```
or
```python
reviews_written = reviews.groupby('taster_twitter_handle').taster_twitter_handle.count()
```


In [ ]:
#q1.hint()
#q1.solution()

## 2.
What is the best wine I can buy for a given amount of money? Create a `Series` whose index is wine prices and whose values is the maximum number of points a wine costing that much was given in a review. Sort the values by price, ascending (so that `4.0` dollars is at the top and `3300.0` dollars is at the bottom).

In [ ]:

best_rating_per_price = reviews.groupby('price')['points'].max().sort_index()

# Check your answer
q2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# q2.hint()
# q2.solution()

## 3.
What are the minimum and maximum prices for each `variety` of wine? Create a `DataFrame` whose index is the `variety` category from the dataset and whose values are the `min` and `max` values thereof.

In [ ]:
price_extremes = reviews.groupby('variety').price.agg([min,max])

# Check your answer
q3.check()

/tmp/ipykernel_33/533615308.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  price_extremes = reviews.groupby('variety').price.agg([min,max])
/tmp/ipykernel_33/533615308.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  price_extremes = reviews.groupby('variety').price.agg([min,max])


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# q3.hint()
# q3.solution()

## 4.
What are the most expensive wine varieties? Create a variable `sorted_varieties` containing a copy of the dataframe from the previous question where varieties are sorted in descending order based on minimum price, then on maximum price (to break ties).

In [ ]:
sorted_varieties = price_extremes.sort_values(by=['min', 'max'], ascending=False)

# Check your answer
q4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# q4.hint()
# q4.solution()

## 5.
Create a `Series` whose index is reviewers and whose values is the average review score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.

In [ ]:
reviewer_mean_ratings = reviews.groupby('taster_name').points.mean()

# Check your answer
q5.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
#q5.hint()
#q5.solution()

Are there significant differences in the average scores assigned by the various reviewers? Run the cell below to use the `describe()` method to see a summary of the range of values.

In [ ]:
reviewer_mean_ratings.describe()

count    19.000000
mean     88.233026
std       1.243610
min      85.855422
25%      87.323501
50%      88.536235
75%      88.975256
max      90.562551
Name: points, dtype: float64

## 6.
What combination of countries and varieties are most common? Create a `Series` whose index is a `MultiIndex`of `{country, variety}` pairs. For example, a pinot noir produced in the US should map to `{"US", "Pinot Noir"}`. Sort the values in the `Series` in descending order based on wine count.

In [ ]:
country_variety_counts = reviews.groupby(['country', 'variety']).size().sort_values(ascending=False)

# Check your answer
q6.check()

<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> Incorrect value for `country_variety_counts`

In [ ]:
# q6.hint()
# q6.solution()

# Keep going

Move on to the [**data types and missing data**](https://www.kaggle.com/residentmario/data-types-and-missing-values).

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*